test_data

In [28]:
import pandas as pd

coupang_3p = pd.read_excel("./test_data/(전달용) 쿠팡_리뷰 20240223 복사본.xlsx", sheet_name="(RAW) 쿠팡_3P")
coupang_rocket = pd.read_excel("./test_data/(전달용) 쿠팡_리뷰 20240223 복사본.xlsx", sheet_name="(RAW) 쿠팡_로켓")

In [46]:
# 3P 데이터에서 "등록된 내용이 없습니다" 제외하고 컬럼명 변경
coupang_3p_cleaned = coupang_3p[coupang_3p['상품평 코멘트'] != '등록된 내용이 없습니다.'].rename(columns={'상품평 코멘트': '리뷰내용'})

# 로켓 데이터에서 "등록된 내용이 없습니다" 제외
coupang_rocket_cleaned = coupang_rocket[coupang_rocket['리뷰내용'] != '등록된 내용이 없습니다.']

# 정제된 데이터 합치기
combined_reviews = pd.concat([coupang_3p_cleaned, coupang_rocket_cleaned], ignore_index=True)

combined_reviews = combined_reviews.dropna(subset='리뷰내용')

# 결과 확인
print(f"정제 후 총 리뷰 수: {len(combined_reviews)}")
print(f"정제 후 3P 리뷰 수: {len(coupang_3p_cleaned)}")
print(f"정제 후 로켓 리뷰 수: {len(coupang_rocket_cleaned)}")

정제 후 총 리뷰 수: 3083
정제 후 3P 리뷰 수: 2060
정제 후 로켓 리뷰 수: 1053


In [65]:
test_data = combined_reviews['리뷰내용'][:1000]

In [66]:
# 줄바꿈으로 구분하여 합치기
combined_text_newline = '\n'.join(test_data)

In [67]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# trie_chat_model
from langchain.chat_models import ChatOpenAI

# OutputParser
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field
from typing import List

In [3]:
swot_test_llm = ChatOpenAI(
    temperature=0.5,  # 창의성
    model_name="gpt-3.5-turbo",  # 모델명
)

/var/folders/m1/bqb2x_hs2p3c9xfm8sbxkgcr0000gn/T/ipykernel_75698/960742898.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  swot_test_llm = ChatOpenAI(


In [84]:
class SWOTKeywords(BaseModel):
    strength_keyword: str = Field(description="제품의 경쟁 우위를 나타내는 긍정적인 키워드나 문구")
    weakness_keyword: str = Field(description="제품의 한계점이나 개선이 필요한 부분을 나타내는 키워드나 문구")
    opportunity_keyword: str = Field(description="리뷰를 통해 발견된 잠재적 성장 기회나 새로운 시장 기회를 나타내는 키워드나 문구")
    threat_keyword: str = Field(description="제품의 성공을 저해할 수 있는 우려사항이나 외부 위험 요소를 나타내는 키워드나 문구")

class SWOTResponse(BaseModel):
    keywords: List[SWOTKeywords] = Field(description="리뷰 텍스트에서 SWOT 카테고리별로 추출된 키워드 리스트")

In [85]:
# 2. Parser 설정
swot_parser = PydanticOutputParser(pydantic_object=SWOTResponse)

In [86]:
swot_template = """
    다음 리뷰 텍스트를 분석하여 SWOT 분석을 수행하고, 정확히 아래 JSON 형식으로 출력해주세요.
    분석이 어려운 카테고리가 있더라도 반드시 모든 필드를 포함해야 합니다.
    키워드나 인사이트가 없는 경우 "없음" 또는 "관련 내용 없음"으로 표시해주세요.

    분석할 리뷰 텍스트:
    {review_text}

    반드시 다음 형식의 JSON으로 출력해주세요:
    {{
        "keywords": [
            {{
                "strength_keyword": "강점 키워드1",
                "weakness_keyword": "약점 키워드1",
                "opportunity_keyword": "기회 키워드1",
                "threat_keyword": "위협 키워드1"
            }},
            {{
                "strength_keyword": "강점 키워드2",
                "weakness_keyword": "약점 키워드2",
                "opportunity_keyword": "기회 키워드2",
                "threat_keyword": "위협 키워드2"
            }}
        ]
    }}

    주의사항:
    1. 반드시 위의 JSON 형식을 정확히 따라주세요
    2. 모든 필드는 필수이며, 값이 없는 경우 "없음"으로 표시
    3. 다른 설명이나 부가 텍스트 없이 JSON만 출력
    4. 실제 키워드는 한글로 작성

    {format_instructions}
    """

In [87]:
swot_prompt = PromptTemplate(
    template=swot_template,
    input_variables=["review_text"],
    partial_variables={"format_instructions": swot_parser.get_format_instructions()}
)

In [88]:
swot_chain = (
    swot_prompt
    | swot_test_llm
    | swot_parser
)

In [90]:
swot_response = swot_chain.invoke({"review_text": combined_text_newline})

In [95]:
def collect_swot_keywords(swot_response):
    # 각 카테고리별 키워드를 저장할 딕셔너리
    collected = {
        'strengths': [],
        'weaknesses': [],
        'opportunities': [],
        'threats': []
    }
    
    # 각 키워드 세트에서 카테고리별로 수집
    for keyword_set in swot_response.keywords:
        # 강점 수집
        if keyword_set.strength_keyword and keyword_set.strength_keyword != "없음":
            # 쉼표로 구분된 경우 분리
            strengths = [s.strip() for s in keyword_set.strength_keyword.split(',')]
            collected['strengths'].extend(strengths)
        
        # 약점 수집
        if keyword_set.weakness_keyword and keyword_set.weakness_keyword != "없음":
            weaknesses = [w.strip() for w in keyword_set.weakness_keyword.split(',')]
            collected['weaknesses'].extend(weaknesses)
            
        # 기회 수집
        if keyword_set.opportunity_keyword and keyword_set.opportunity_keyword != "없음":
            opportunities = [o.strip() for o in keyword_set.opportunity_keyword.split(',')]
            collected['opportunities'].extend(opportunities)
            
        # 위협 수집
        if keyword_set.threat_keyword and keyword_set.threat_keyword != "없음":
            threats = [t.strip() for t in keyword_set.threat_keyword.split(',')]
            collected['threats'].extend(threats)
    
    # 중복 제거
    collected = {k: list(set(v)) for k, v in collected.items()}
    
    return collected

# 사용 예시
result = collect_swot_keywords(swot_response)

print("\n=== SWOT 분석 결과 ===")
print("\n강점:")
for strength in result['strengths']:
    print(f"- {strength}")

print("\n약점:")
for weakness in result['weaknesses']:
    print(f"- {weakness}")

print("\n기회:")
for opportunity in result['opportunities']:
    print(f"- {opportunity}")

print("\n위협:")
for threat in result['threats']:
    print(f"- {threat}")


=== SWOT 분석 결과 ===

강점:
- 하루 한 알 복용으로 편리함
- 알 크기가 작음
- 가격이 훨씬 착함
- 개별 포장으로 위생적임
- 가격이 합리적임

약점:

기회:
- 온라인 구매 시 추가 할인 제공 등 프로모션 전략 개발 가능성
- 개별 포장된 제품의 위생적 장점을 마케팅 포인트로 활용 가능
- 건강보조식품 시장에서 젊은 층을 타겟팅한 광고 전략 개발 가능성

위협:
- 기존 브랜드에 대한 충성도 있는 고객층 존재 가능성
- 경쟁사의 유사한 저렴한 제품 존재 가능성
- 무료 샘플이나 경쟁사의 프로모션 활동


In [97]:
result

{'strengths': ['하루 한 알 복용으로 편리함',
  '알 크기가 작음',
  '가격이 훨씬 착함',
  '개별 포장으로 위생적임',
  '가격이 합리적임'],
 'weaknesses': [],
 'opportunities': ['온라인 구매 시 추가 할인 제공 등 프로모션 전략 개발 가능성',
  '개별 포장된 제품의 위생적 장점을 마케팅 포인트로 활용 가능',
  '건강보조식품 시장에서 젊은 층을 타겟팅한 광고 전략 개발 가능성'],
 'threats': ['기존 브랜드에 대한 충성도 있는 고객층 존재 가능성',
  '경쟁사의 유사한 저렴한 제품 존재 가능성',
  '무료 샘플이나 경쟁사의 프로모션 활동']}